<a href="https://colab.research.google.com/github/PrabhatGhm7/Spacy_Question_Generation/blob/main/Fill_in_and_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import spacy
import random
#Pre trained English language
nlp = spacy.load("en_core_web_sm")

# Fill in the blanks

In [80]:
def generate_fill_in_the_blank(sentence, blank_type=None):
    doc = nlp(sentence)

    elements = {
        "subject": None,
        "verb": None,
        "object": None,
        "place": None,
        "time": None,
        "reason": None,
        "manner": None
    }

    for token in doc:
        if token.dep_ == "nsubj":
            elements["subject"] = token.text

        elif token.pos_ == "VERB":
            elements["verb"] = token.text

        elif token.dep_ in ["dobj", "attr"]:
            elements["object"] = token.text

        #creating subtree for multiple elements
        elif token.dep_ == "pobj" or token.ent_type_ in ["GPE", "LOC", "ORG"]:
            elements["place"] = " ".join([t.text for t in token.subtree])

        elif token.ent_type_ in ["DATE", "TIME"]:
            elements["time"] = " ".join([t.text for t in token.subtree])

        elif token.dep_ in ["prep"] and token.text in ["because", "due"]:
            elements["reason"] = " ".join([t.text for t in token.subtree])

        elif token.dep_ == "advmod":
            elements["manner"] = token.text

    #if blank type is provide then use it
    if blank_type and blank_type in elements and elements[blank_type]:
        blank_word = elements[blank_type]

    else: # if blank type is not provided make list in key value and then use one of them to fill
        available_blanks = [(k, v) for k, v in elements.items() if v]

        if not available_blanks:
            return "Can't Generate Fill-in-the-Blanks Question"

        blank_type, blank_word = random.choice(available_blanks)

    blank_sentence = sentence.replace(blank_word, "_____",random.randint(0,2))
    return f"Fill in the blank : {blank_sentence} --({blank_type})"

In [81]:
#Trying
print(generate_fill_in_the_blank("Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, 1879.", "time"))

print(generate_fill_in_the_blank("Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, 1879.", "verb"))

print(generate_fill_in_the_blank("The dog is walking quickly in the bridge because it is happy.", "manner"))

print(generate_fill_in_the_blank("Six weeks later the family moved to Munich, where he later on began his education at the Luitpold, Gymnasium.", "object"))

Fill in the blank : Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, _____. --(time)
Fill in the blank : Albert Einstein was _____ at Ulm, in Württemberg, Germany, on March 14, 1879. --(verb)
Fill in the blank : The dog is walking _____ in the bridge because it is happy. --(manner)
Fill in the blank : Six weeks later the family moved to Munich, where he later on began his _____ at the Luitpold, Gymnasium. --(object)


# Match the following


In [82]:
def generate_match_the_following(sentence):
    """We Generates match the following question from the given sentence,
    by extracting grammatical components and pairing them with their roles. """

    doc = nlp(sentence)
    parts = {}

    # Extract components from the sentence.
    for token in doc:
        if token.dep_ == "nsubj" and "Subject" not in parts:
            parts["Subject"] = token.text

        elif token.dep_ == "ROOT" and "Verb" not in parts:
            parts["Verb"] = token.lemma_

        elif token.dep_ in ["dobj", "attr"] and "Object" not in parts:
            parts["Object"] = token.text

        elif (token.dep_ == "pobj" or token.ent_type_ in ["GPE", "LOC", "ORG"]) and "Place" not in parts:
            parts["Place"] = token.text

        elif token.ent_type_ in ["DATE", "TIME"] and "Time" not in parts:
            parts["Time"] = token.text

    if not parts:
        return "Not enough info"

    #Lists of items and their roles.
    items = list(parts.values())
    roles = list(parts.keys())


    #Shuffling the Roles
    shuffled_roles = roles[:] #Create a copy of role
    random.shuffle(shuffled_roles)

    #match the following generation
    question_text = "Match the following (items with grammar roles:)\n\n"
    question_text += "Items:\n"

    #iterates over the list items, return each element along with an index.
    for idx, item in enumerate(items, 1):
        question_text += f"  {idx}. {item}\n"

    question_text += "\n Options:\n"
    for i, role in enumerate(shuffled_roles):
        option_letter = chr(65 + i)
        question_text += f"  {option_letter}. {role}\n"

    return question_text

In [83]:
#trying

print(generate_match_the_following("Six weeks later the family moved to Munich, where he later on began his education at the Luitpold, Gymnasium."))

Match the following (items with grammar roles:)

Items:
  1. Six
  2. family
  3. move
  4. Munich
  5. education

Options:
  A. Object
  B. Verb
  C. Subject
  D. Place
  E. Time



In [84]:

print(generate_match_the_following("The quick brown fox jumps over the lazy dog."))

Match the following (items with grammar roles:)

Items:
  1. fox
  2. jump
  3. dog

Options:
  A. Verb
  B. Place
  C. Subject



In [85]:
print(generate_match_the_following("Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, 1879."))

Match the following (items with grammar roles:)

Items:
  1. bear
  2. Ulm
  3. March

Options:
  A. Verb
  B. Place
  C. Time

